In [12]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from matplotlib.colors import Normalize
import matplotlib.cm as cm

# Constants
NUM_PARTICLES = 1000
TIME_STEP = 0.1
PARTICLE_RADIUS = 0.2  # Reduced particle radius
CIRCLE_CENTER = np.array([0, 0])
CIRCLE_RADIUS = 5
SQUARE_BOUNDARY = 10  # Half-width and half-height of the square
FRAME_COUNT = 200

# Initialize particles
particles = {
    'position': np.random.rand(NUM_PARTICLES, 2) * 20 - 10,
    'velocity': np.random.randn(NUM_PARTICLES, 2)
}

# Normalize for color mapping
norm = Normalize()
color_map = cm.plasma

def update_particles(particles):
    # Update particle positions
    particles['position'] += particles['velocity'] * TIME_STEP

    # Particle-to-particle collision
    for i in range(NUM_PARTICLES):
        for j in range(i + 1, NUM_PARTICLES):
            dist = np.linalg.norm(particles['position'][i] - particles['position'][j])
            if dist < 2 * PARTICLE_RADIUS:
                # Calculate collision response
                direction = (particles['position'][i] - particles['position'][j]) / dist
                v1 = particles['velocity'][i]
                v2 = particles['velocity'][j]
                particles['velocity'][i] = v1 - np.dot(v1 - v2, direction) * direction
                particles['velocity'][j] = v2 + np.dot(v1 - v2, direction) * direction

                # Separate overlapping particles
                overlap = 2 * PARTICLE_RADIUS - dist
                particles['position'][i] += direction * overlap / 2
                particles['position'][j] -= direction * overlap / 2

    # Particle-to-circle collision
    for i in range(NUM_PARTICLES):
        dist_to_center = np.linalg.norm(particles['position'][i] - CIRCLE_CENTER)
        if dist_to_center < CIRCLE_RADIUS + PARTICLE_RADIUS:
            # Reflect velocity
            direction = (particles['position'][i] - CIRCLE_CENTER) / dist_to_center
            particles['velocity'][i] -= 2 * np.dot(particles['velocity'][i], direction) * direction
            # Reposition particle outside the circle to prevent overlap
            overlap = CIRCLE_RADIUS + PARTICLE_RADIUS - dist_to_center
            particles['position'][i] += direction * overlap

    # Particle-to-square boundary collision
    for i in range(NUM_PARTICLES):
        for dim in range(2):  # Check for both x and y dimensions
            if abs(particles['position'][i, dim]) > SQUARE_BOUNDARY - PARTICLE_RADIUS:
                # Reflect velocity
                particles['velocity'][i, dim] *= -1
                # Reposition particle inside the boundary to prevent overlap
                sign = -1 if particles['position'][i, dim] < 0 else 1
                overlap = abs(particles['position'][i, dim]) - (SQUARE_BOUNDARY - PARTICLE_RADIUS)
                particles['position'][i, dim] -= sign * overlap

def animate(frame):
    update_particles(particles)

    plt.cla()
    plt.gca().set_facecolor('#4b4b4b')  # Dark grey plot area
    fig.patch.set_facecolor('#4b4b4b')  # Dark grey figure background
    circle = plt.Circle(CIRCLE_CENTER, CIRCLE_RADIUS, color='r', fill=False)
    plt.gca().add_patch(circle)
    
    # Calculate speeds and map to colors
    speeds = np.linalg.norm(particles['velocity'], axis=1)
    colors = color_map(norm(speeds))

    plt.scatter(particles['position'][:, 0], particles['position'][:, 1], s=5, color=colors)  # Smaller particle size
    plt.xlim(-SQUARE_BOUNDARY, SQUARE_BOUNDARY)
    plt.ylim(-SQUARE_BOUNDARY, SQUARE_BOUNDARY)
    plt.gca().set_aspect('equal', adjustable='box')

# Create animation
fig = plt.figure()
ani = FuncAnimation(fig, animate, frames=FRAME_COUNT, interval=100)

# Save the animation as a GIF
ani.save('particle_animation.gif', writer=PillowWriter(fps=20))
